In [2]:
# 挂载 google 云盘
from google.colab import drive
drive.mount('/content/gdrive')

# 将文件路径指定为数据路径
import os
os.chdir('/content/gdrive/My Drive/comment')

import numpy as np
import pandas as pd

from tqdm import tqdm #显示运行进度条

# # 画图库
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# import seaborn as sb
# import matplotlib.pyplot as plt

# 文字处理库
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
# nltk.download('stopwords') #下载停止词
import string
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Using TensorFlow backend.


In [3]:
nltk.download('stopwords') #下载停止词
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
labels = train_data.columns[2:].tolist()
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
all_comments = pd.concat([train_data['comment_text'], test_data['comment_text']])
stop_words = set(stopwords.words('english'))
stop_words.update(string.punctuation)
comments_refined = []
for line in tqdm(all_comments):
    comments_refined.append(' '.join([word for word in tokenizer.tokenize(line) if word not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 312735/312735 [00:10<00:00, 30463.12it/s]


In [18]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
    max_features=50000).fit(comments_refined)
train_features, test_features = map(lambda x: vectorizer.transform(x), [train_data['comment_text'], test_data['comment_text']] )
scores = []
submission = pd.DataFrame.from_dict({'id': test_data['id']})
for label in tqdm(labels):
    classifier = LogisticRegression(C=4, 
                                    solver='sag')
    cv_score = np.mean(cross_val_score(classifier, 
                                       train_features, 
                                       train_data[label], 
                                       cv=3, 
                                       scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(label, cv_score))

    classifier.fit(train_features, train_data[label])
    submission[label] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
submission.to_csv('submission_simple.csv', index=False)





  0%|          | 0/6 [00:00<?, ?it/s]

CV score for class toxic is 0.973382006386684






 17%|█▋        | 1/6 [00:20<01:41, 20.25s/it]

CV score for class severe_toxic is 0.9832314692514882






 33%|███▎      | 2/6 [00:44<01:25, 21.34s/it]

...

CV score for class insult is 0.9775646210344795






 83%|████████▎ | 5/6 [01:45<00:20, 20.94s/it]

CV score for class identity_hate is 0.9737676642141246






100%|██████████| 6/6 [02:06<00:00, 21.05s/it]





Total CV score is 0.9798070920140471


In [0]:

#   0%|          | 0/6 [00:00<?, ?it/s]CV score for class toxic is 0.973382006386684
#  17%|█▋        | 1/6 [00:20<01:41, 20.25s/it]CV score for class severe_toxic is 0.9832314692514882
#  33%|███▎      | 2/6 [00:44<01:25, 21.34s/it]CV score for class obscene is 0.9850783263974195
#  50%|█████     | 3/6 [01:03<01:02, 20.81s/it]CV score for class threat is 0.9858184648000871
#  67%|██████▋   | 4/6 [01:23<00:40, 20.47s/it]CV score for class insult is 0.9775646210344795
#  83%|████████▎ | 5/6 [01:45<00:20, 20.94s/it]CV score for class identity_hate is 0.9737676642141246
# 100%|██████████| 6/6 [02:06<00:00, 21.05s/it]
# Total CV score is 0.9798070920140471
